# Data Cleaning

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from typing import Tuple
import json
import os

### Exploratory Analysis

In [2]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

df_train.head()

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,...,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,...,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,...,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,...,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,...,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1235 non-null   int64  
 1   surgery                1235 non-null   object 
 2   age                    1235 non-null   object 
 3   hospital_number        1235 non-null   int64  
 4   rectal_temp            1235 non-null   float64
 5   pulse                  1235 non-null   float64
 6   respiratory_rate       1235 non-null   float64
 7   temp_of_extremities    1196 non-null   object 
 8   peripheral_pulse       1175 non-null   object 
 9   mucous_membrane        1214 non-null   object 
 10  capillary_refill_time  1229 non-null   object 
 11  pain                   1191 non-null   object 
 12  peristalsis            1215 non-null   object 
 13  abdominal_distention   1212 non-null   object 
 14  nasogastric_tube       1155 non-null   object 
 15  naso

In [4]:
# Changing Data Types
df_train = df_train.astype({"hospital_number": 'category'})

In [5]:
print(f"Before dropping null values: {len(df_train)}\nAfter dropping null values: {len(df_train.dropna(axis=0, how='any'))}")

Before dropping null values: 1235
After dropping null values: 771


> We can consider dropping all rows with null values but I feel we may need to take a hybrid approach dropping some rows and columns (devil is in the details)

In [6]:
df_train.describe()

,id,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
count,1235.0000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000
mean,617.0000,38.202186,79.574089,30.054251,4.382591,49.602429,21.388016,3.290931,3832.496356,14.612146,3.577328
std,356.6581,0.788668,29.108638,16.452066,1.937357,10.535800,26.676453,1.589195,5436.733774,193.705735,88.858953
min,0.0000,35.400000,30.000000,8.000000,1.000000,23.000000,3.500000,0.100000,0.000000,0.000000,0.000000
25%,308.5000,37.800000,53.000000,18.000000,2.000000,43.000000,6.600000,2.000000,2205.000000,0.000000,0.000000
50%,617.0000,38.200000,76.000000,28.000000,4.500000,48.000000,7.500000,3.000000,2209.000000,0.000000,0.000000
75%,925.5000,38.600000,100.000000,36.000000,6.000000,57.000000,9.100000,4.300000,3205.000000,0.000000,0.000000
max,1234.0000,40.800000,184.000000,96.000000,7.500000,75.000000,89.000000,10.100000,41110.000000,3112.000000,2209.000000


In [7]:
df_train.nunique()

id                       1235
surgery                     2
age                         2
hospital_number           255
rectal_temp                43
pulse                      50
respiratory_rate           37
temp_of_extremities         4
peripheral_pulse            4
mucous_membrane             6
capillary_refill_time       3
pain                        6
peristalsis                 5
abdominal_distention        4
nasogastric_tube            3
nasogastric_reflux          4
nasogastric_reflux_ph      26
rectal_exam_feces           5
abdomen                     5
packed_cell_volume         49
total_protein              83
abdomo_appearance           3
abdomo_protein             54
surgical_lesion             2
lesion_1                   57
lesion_2                    4
lesion_3                    2
cp_data                     2
outcome                     3
dtype: int64

In [8]:
categorical_cols = list(set(df_train.columns) - set(df_train._get_numeric_data().columns))
print(f"CATEGORIES IN COLUMNS\n---------------------------------------------------------------------------\n")
print("Column \t\t\t\t Unique Categories")
for col in categorical_cols:
    print(f"{col} \t\t {list(df_train[col].unique())}")

CATEGORIES IN COLUMNS
---------------------------------------------------------------------------

Column 				 Unique Categories
abdomo_appearance 		 ['serosanguious', 'cloudy', 'clear', nan]
hospital_number 		 [530001, 533836, 529812, 5262541, 5299629, 529642, 534787, 529461, 528742, 529640, 528682, 530028, 528548, 528134, 528305, 534885, 5290482, 5279822, 533692, 535208, 528523, 529893, 534145, 530233, 529399, 530354, 528503, 529796, 527916, 530360, 528298, 533871, 529388, 527563, 534163, 529827, 535196, 535176, 529045, 527518, 527463, 529172, 528996, 533887, 528904, 535407, 533902, 523190, 534073, 534135, 5290409, 529160, 534917, 534784, 5299253, 534004, 534115, 529667, 5297159, 529427, 527677, 530612, 535415, 530561, 530242, 530002, 5289419, 529498, 529126, 5291409, 5287179, 530526, 5290759, 532110, 534293, 534280, 528214, 527933, 5283431, 528743, 529766, 529304, 530401, 527702, 529849, 534157, 534998, 529340, 530276, 527927, 534886, 527365, 528641, 528461, 528469, 529607, 533942, 

> 1. Hospital number needs to replaced by something in feature engineering
> 2. There are many categories here that can use sequential encoding instead of one hot encoding but we need to know their starting index and order (also value for null values)

In [16]:
with open("data/Regular_Processed/With_Ordinal_Encoding/Pos_and_Neg/encoding.json", "r") as json_file:
    ENCODINGS = json.load(json_file)

print(ENCODINGS)

{'COLUMNS_TO_BE_DROPPED': ['id', 'hospital_number'], 'BINARY_COLUMNS': ['age', 'surgical_lesion', 'surgery', 'cp_data'], 'NUMERICAL_COLUMNS': ['rectal_temp', 'pulse', 'respiratory_rate', 'nasogastric_reflux_ph', 'packed_cell_volume', 'total_protein', 'abdomo_protein', 'lesion_1', 'lesion_2', 'lesion_3'], 'MULTI_TYPE_COLUMNS': ['abdomen', 'mucous_membrane'], 'ORDINAL_TYPE_COLUMNS': {'temp_of_extremities': {'cold': -2, 'cool': -1, 'normal': 0, 'warm': 1, 'nan': -1}, 'peripheral_pulse': {'absent': -2, 'reduced': -1, 'normal': 0, 'increased': 1, 'nan': -1}, 'capillary_refill_time': {'less_3_sec': -1, '3': 0, 'more_3_sec': 1, 'nan': -1}, 'pain': {'alert': 0, 'depressed': 1, 'slight': 2, 'moderate': 3, 'mild_pain': 4, 'severe_pain': 5, 'extreme_pain': 6, 'nan': 1}, 'peristalsis': {'hypermotile': -2, 'normal': -1, 'hypomotile': 0, 'absent': 1, 'nan': 0, 'distend_small': 2}, 'abdominal_distention': {'none': 0, 'slight': 1, 'moderate': 2, 'severe': 3, 'nan': 2}, 'nasogastric_tube': {'none': 0, 

In [17]:
COLUMNS_TO_BE_DROPPED = ENCODINGS['COLUMNS_TO_BE_DROPPED']
BINARY_COLUMNS = ENCODINGS['BINARY_COLUMNS']
NUMERICAL_COLUMNS = ENCODINGS['NUMERICAL_COLUMNS']
MULTI_TYPE_COLUMNS = ENCODINGS["MULTI_TYPE_COLUMNS"]
ORDINAL_TYPE_COLUMNS = ENCODINGS["ORDINAL_TYPE_COLUMNS"]
TARGET = ["outcome"]

### Data Formatting
This includes one-hot encoding, binary encoding, and ordinal encoding

In [18]:
def _preprocess_num_data(train_df: pd.DataFrame, test_df: pd.DataFrame, NUMERICAL_COLUMNS: list):
    drop_cols = list(set(train_df.columns) - set(NUMERICAL_COLUMNS))
    train_df_temp = train_df.drop(drop_cols, axis=1)
    test_df_temp = test_df.drop(drop_cols, axis=1)
    train_arr = train_df_temp.reindex(NUMERICAL_COLUMNS, axis=1).to_numpy()
    test_arr = test_df_temp.reindex(NUMERICAL_COLUMNS, axis=1).to_numpy()

    scaler = StandardScaler()
    train_arr_scaled = scaler.fit_transform(train_arr)
    test_arr_scaled = scaler.transform(test_arr)

    train_df_scaled = pd.DataFrame(data=train_arr_scaled, columns=NUMERICAL_COLUMNS)
    test_df_scaled = pd.DataFrame(data=test_arr_scaled, columns=NUMERICAL_COLUMNS)

    return train_df_scaled, test_df_scaled

def _preprocess_cat_data(df: pd.DataFrame, BINARY_COLUMNS: list, MULTI_TYPE_COLUMNS: list, ORDINAL_TYPE_COLUMNS: dict=None, possible_values_dict: dict=None, type_limit=10) -> pd.DataFrame:
    """Preprocesses categorical data.
    ### Parameters
    possible_values_dict - Contains the possible values of columns for consistant encoding in train and test. (None if training data is being sent)
    ### Returns
    (df, possible_values_dict) for training data\n  
    (df, None) for test data
    """
    def get_possible_values(df_col: pd.Series) -> list:
        """Returns the possible values of a categorical column (removes nan/null values)"""
        possible_values = list(df_col.unique())
        possible_values_2 = []
        for pv in possible_values:
            if not pd.isna(pv):
                possible_values_2.append(pv)
        return possible_values_2
    
    if ORDINAL_TYPE_COLUMNS is None:
        ORDINAL_TYPE_COLUMNS = {}

    # Conflict Checking
    ord_cols_set, mult_cols_set, bin_cols_set = set(ORDINAL_TYPE_COLUMNS.keys()), set(MULTI_TYPE_COLUMNS), set(BINARY_COLUMNS)
    if not ord_cols_set.isdisjoint(mult_cols_set):
        raise Exception(f"Data Conflict: ORDINAL_TYPE_COLUMNS and MULTI_TYPE_COLUMNS both contain columns {ord_cols_set.intersection(mult_cols_set)}")
    elif not ord_cols_set.isdisjoint(bin_cols_set):
        raise Exception(f"Data Conflict: ORDINAL_TYPE_COLUMNS and BINARY_COLUMNS both have contain columns {ord_cols_set.intersection(bin_cols_set)}")
    elif not mult_cols_set.isdisjoint(bin_cols_set):
        raise Exception(f"Data Conflict: MULTI_TYPE_COLUMNS and BINARY_COLUMNS both have contain columns {mult_cols_set.intersection(bin_cols_set)}")
    
    # Find the correct mode
    if possible_values_dict is None:
        train_mode = True
        possible_values_dict = {}
    else:
        train_mode = False
    
    # 2) Binary columns
    for bc in BINARY_COLUMNS:
        if df[bc].nunique() != 2:
            raise Exception(f"Binary Column {bc} is not binary it has {df[bc].nunique()} possible values.")
        possible_values = list(df[bc].unique()) if train_mode else possible_values_dict[bc]
        if train_mode:
            possible_values_dict[bc] = possible_values
        value_mapping = {possible_values[0]: 1, possible_values[1]: 0}
        new_col_name = f"{bc}_is_{possible_values[0]}"
        new_col = [value_mapping[val] for val in df[bc]]
        df[new_col_name] = new_col
    df = df.drop(BINARY_COLUMNS, axis=1)
    
    # 3) MULTI TYPE COLUMNS
    for mc in MULTI_TYPE_COLUMNS:
        if df[mc].nunique() > type_limit:
            raise Exception(f"Column {mc} has {df[mc].nunique()} values which exceeds the type limit of {type_limit}")
        possible_values = get_possible_values(df[mc]) if train_mode else possible_values_dict[mc]
        if train_mode:
            possible_values_dict[mc] = possible_values
        value_mapping = {possible_values[i]: i for i in range(len(possible_values))}
        new_col_names = [f"{mc}_is_{pv}" for pv in possible_values]
        new_cols = np.zeros((len(possible_values), len(df)), dtype=np.int8)
        col = list(df[mc])
        for i in range(len(col)):
            # For null and unseen values will be zero for all columns
            if not(pd.isna(col[i])) and (col[i] in value_mapping):
                new_cols[value_mapping[col[i]]][i] = 1

        for i in range(len(possible_values)):
            df[new_col_names[i]] = new_cols[i]
    df = df.drop(MULTI_TYPE_COLUMNS, axis=1)

    # 4) ORDINAL COLUMNS
    for oc, value_mapping in ORDINAL_TYPE_COLUMNS.items():
        null_value = value_mapping['nan']
        col = list(df[oc])
        new_col = []
        for i in range(len(col)):
            if pd.isna(col[i]):
                new_col.append(null_value)
            elif col[i] in value_mapping:
                new_col.append(value_mapping[col[i]])
            else:
                raise Exception(f"Unknown Value: {col[i]} found in column {oc}")
        df[oc] = new_col


    return df, possible_values_dict

def preprocess_data(train_df: pd.DataFrame, test_df: pd.DataFrame, COLUMNS_TO_BE_DROPPED: list,BINARY_COLUMNS: list, MULTI_TYPE_COLUMNS: list, NUMERICAL_COLUMNS: list, ORDINAL_TYPE_COLUMNS: dict = None, type_limit=10):
    """Handles all data preprocessing including One Hot Encoding and Standrad Scalar
    ### Returns
    (train_df, test_df) as pd.Dataframes after the processing
    ### Note
    Please remove target column from train data before processing, the dataframes need to have the same columns"""
    # Check
    if set(train_df.columns) != set(test_df.columns):
        raise Exception("Train and Test data should have the same columns. (Please remove target from train data)")
    
    # 1) Drop unwanted columns
    train_df = train_df.drop(COLUMNS_TO_BE_DROPPED, axis=1)
    test_df = test_df.drop(COLUMNS_TO_BE_DROPPED, axis=1)

    # 2) PreProcess categorical data
    train_df, possible_values_dict = _preprocess_cat_data(df=train_df, BINARY_COLUMNS=BINARY_COLUMNS, MULTI_TYPE_COLUMNS=MULTI_TYPE_COLUMNS, ORDINAL_TYPE_COLUMNS=ORDINAL_TYPE_COLUMNS, type_limit=type_limit, possible_values_dict=None)
    test_df, _ = _preprocess_cat_data(df=test_df, BINARY_COLUMNS=BINARY_COLUMNS, MULTI_TYPE_COLUMNS=MULTI_TYPE_COLUMNS, ORDINAL_TYPE_COLUMNS=ORDINAL_TYPE_COLUMNS, type_limit=type_limit, possible_values_dict=possible_values_dict)

    # 3) Preprocess numerical data
    train_df_scaled, test_df_scaled = _preprocess_num_data(train_df= train_df, test_df=test_df, NUMERICAL_COLUMNS=NUMERICAL_COLUMNS)
    for col in NUMERICAL_COLUMNS:
        train_df[col] = train_df_scaled[col]
        test_df[col] = test_df_scaled[col]

    return train_df, test_df

### Normal Data Splitting

In [19]:
# Do data preprocessing
train_df_processed, test_df_processed = preprocess_data(train_df=df_train.drop(['outcome'], axis=1), test_df=df_test, COLUMNS_TO_BE_DROPPED=COLUMNS_TO_BE_DROPPED, BINARY_COLUMNS=BINARY_COLUMNS, MULTI_TYPE_COLUMNS=MULTI_TYPE_COLUMNS, NUMERICAL_COLUMNS=NUMERICAL_COLUMNS, ORDINAL_TYPE_COLUMNS=ORDINAL_TYPE_COLUMNS)

# Handle outcome (target)
possible_outcome_values = list(df_train['outcome'].unique())
outcome_value_dict = {possible_outcome_values[i] : i for i in range(len(possible_outcome_values))}
train_df_processed['outcome'] = [outcome_value_dict[list(df_train['outcome'])[i]] for i in range(len(df_train['outcome']))]

# Do train valid split (there is a bug with scikit-learn which is causing a lot of NaN values to appear in the sparse dataset so we have to resort to this)
train_df, valid_df = train_test_split(train_df_processed, test_size=0.2)

In [20]:
possible_outcome_values

['died', 'euthanized', 'lived']

In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 988 entries, 284 to 350
Data columns (total 36 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   rectal_temp                       988 non-null    float64
 1   pulse                             988 non-null    float64
 2   respiratory_rate                  988 non-null    float64
 3   temp_of_extremities               988 non-null    int64  
 4   peripheral_pulse                  988 non-null    int64  
 5   capillary_refill_time             988 non-null    int64  
 6   pain                              988 non-null    int64  
 7   peristalsis                       988 non-null    int64  
 8   abdominal_distention              988 non-null    int64  
 9   nasogastric_tube                  988 non-null    int64  
 10  nasogastric_reflux                988 non-null    int64  
 11  nasogastric_reflux_ph             988 non-null    float64
 12  rectal_exam

In [22]:
# Convert to arrays
y_train = train_df['outcome'].to_numpy()
y_valid = valid_df['outcome'].to_numpy()

# Reordering columns in train, valid and test dataset before converting to array
column_order = list(train_df.drop(['outcome'], axis=1).columns)
X_train = train_df.drop(['outcome'], axis=1).reindex(column_order, axis=1).to_numpy()
X_valid = valid_df.drop(['outcome'], axis=1).reindex(column_order, axis=1).to_numpy()
X_test = test_df_processed.reindex(column_order, axis=1).to_numpy()

In [24]:
# Exporting the processed data as dataframes
SAVE_DIR = "data/Regular_Processed/With_Ordinal_Encoding/Pos_and_Neg"
train_df.to_csv(os.path.join(SAVE_DIR, 'train_processed.csv'))
valid_df.to_csv(os.path.join(SAVE_DIR, 'valid_processed.csv'))
test_df_processed.to_csv(os.path.join(SAVE_DIR, 'test_processed.csv'))

In [25]:
np.save(os.path.join(SAVE_DIR, 'X_train.npy'), X_train)
np.save(os.path.join(SAVE_DIR, 'X_valid.npy'), X_valid)
np.save(os.path.join(SAVE_DIR, 'X_test.npy'), X_test)
np.save(os.path.join(SAVE_DIR, 'y_train.npy'), y_train)
np.save(os.path.join(SAVE_DIR, 'y_valid.npy'), y_valid)

### Autoencoder Data Splitting
Data splitting for the autoencoder (no y values involved and we can use the test data)

In [16]:
# train_df_processed, test_df_processed
merged_df = pd.concat([train_df_processed, test_df_processed], ignore_index=True)

autoencoder_train, autoencoder_test = train_test_split(merged_df.drop(['outcome'], axis=1), test_size=0.2, shuffle=True)
autoencoder_train_arr = autoencoder_train.to_numpy()
autoencoder_test_arr = autoencoder_test.to_numpy()
print(f"{autoencoder_train_arr.shape}\n{autoencoder_test_arr.shape}")


(1647, 66)
(412, 66)


In [17]:
np.save('data/Autoencoder_processed/Autoencoder_train.npy', autoencoder_train_arr)
np.save('data/Autoencoder_processed/Autoencoder_test.npy', autoencoder_test_arr)